In [29]:
%load_ext autoreload
%autoreload 2
from autoseg.datasets import get_dataset_path
from autoseg.datasets.utils import get_shape, get_voxel_size
import networkx as nx
import zarr
import numpy as np
from funlib.evaluate import (
    rand_voi,
    expected_run_length,
    get_skeleton_lengths,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def get_dataset_stats(dataset):
  z = get_dataset_path(dataset + ".zarr")
  s = get_dataset_path(dataset + ".graphml")
  skeletons = nx.read_graphml(s)


  num_edges = skeletons.number_of_edges()
  num_nodes = skeletons.number_of_nodes()
  num_objects = nx.number_connected_components(skeletons)

  skeleton_lengths = get_skeleton_lengths(
    skeletons,
    skeleton_position_attributes=["position_z", "position_y", "position_x"],
    skeleton_id_attribute="id",
    store_edge_length="length"
  )

  total_length = np.sum([l for _, l in skeleton_lengths.items()])
  average_length = total_length
  std_length = np.std([l for _, l in skeleton_lengths.items()])
  shortest_length = min([l for _, l in skeleton_lengths.items()])
  longest_length = max([l for _, l in skeleton_lengths.items()])

  voxel_size = get_voxel_size(z, "volumes/image/s1")
  shape = get_shape(z, "volumes/image/s1")
  size = [v*s for v, s in zip(voxel_size, shape)]
  volume = size[0] * size[1] * size[2]

In [3]:
dataset = "SynapseWeb/team_dentate/BBCHZ"
z = get_dataset_path(dataset + ".zarr")
s = get_dataset_path(dataset + ".graphml")

In [4]:
skeletons = nx.read_graphml(s)

In [5]:
num_edges = skeletons.number_of_edges()
num_nodes = skeletons.number_of_nodes()

In [6]:
num_objects = nx.number_connected_components(skeletons)

In [7]:
connected_components = nx.connected_components(skeletons)
connected_components = list(connected_components)

In [8]:
cc = connected_components[0]

In [12]:
from funlib.evaluate import (
    rand_voi,
    expected_run_length,
    get_skeleton_lengths,
)

In [13]:
skeleton_lengths = get_skeleton_lengths(
  skeletons,
  skeleton_position_attributes=["position_z", "position_y", "position_x"],
  skeleton_id_attribute="id",
  store_edge_length="length"
)

In [21]:
total_length = np.sum([l for _, l in skeleton_lengths.items()])
average_length = total_length
std_length = np.std([l for _, l in skeleton_lengths.items()])
shortest_length = min([l for _, l in skeleton_lengths.items()])
longest_length = max([l for _, l in skeleton_lengths.items()])
print(total_length, average_length, std_length, shortest_length, longest_length)

412745.794523716 412745.794523716 5456.487510407069 152.16652059555054 24156.008276939392


In [39]:
voxel_size = get_voxel_size(z, "volumes/image/s1")
shape = get_shape(z, "volumes/image/s1")
size = [v*s for v, s in zip(voxel_size, shape)]
volume = size[0] * size[1] * size[2]
print(voxel_size, shape, size, volume)


/home/anton/.cache/autoseg/datasets/SynapseWeb/team_dentate/data/BBCHZ.zarr volumes/image/s1
[50, 4, 4] (50, 2763, 4616) [2500, 11052, 18464] 510160320000


In [43]:
nonzero = np.count_nonzero(image["volumes"]["neuron_ids"]["s1"])
labelled_volume = nonzero * voxel_size[0] * voxel_size[1] * voxel_size[2]
percent_labelled = labelled_volume / volume
print(labelled_volume, percent_labelled * 100)

30469896000 5.972611903646289


In [47]:
unique, counts = np.unique(image["volumes"]["neuron_ids"]["s1"], return_counts=True)

In [57]:
np.count_nonzero(counts > 1000)

87

In [50]:
counts

array([380312430,    209136,     35795,   1318429,    754858,   2143450,
             231,    250313,        13,         1,         1,         1,
               1,         1,         1,         1,         1,        20,
               1,         1,         1,        15,    189122,      5387,
          131098,    697071,         1,         1,         1,        35,
             160,    962210,   3090347,       814,       200,        11,
         2882762,        14,         1,         1,   1219847,    237806,
            1225,        10,    470343,         1,         1,         1,
               2,         2,         2,         3,        11,         1,
              73,         1,        89,         1,    216039,         1,
               1,       261,    266198,     29744,   3293869,    328908,
         1204202,    784447,    825000,    337894,     29981,    395039,
          939838,       201,         1,         1,         1,         9,
               2,         1,         5,         1, 